# BIO559R Example Notebook: Single-Cell Analysis with Python and R

This notebook demonstrates the basic workflow for single-cell RNA-seq analysis using both Python (scanpy) and R (Seurat/ggplot2) in a single Jupyter notebook.

**Author:** BIO559R Tutorial  
**Course:** Introduction to Systems Biology

## 1. Setup and Data Loading

First, let's import the necessary Python libraries and load the R magic extension.

In [ ]:
# Import Python libraries
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configure scanpy settings
sc.settings.verbosity = 3  # verbosity level
sc.settings.set_figure_params(dpi=80, facecolor='white')

print(f"Scanpy version: {sc.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

In [ ]:
# Load R magic extension
%load_ext rpy2.ipython
print("R magic extension loaded successfully!")

## 2. Create Synthetic Single-Cell Data

For this example, we'll create a synthetic single-cell dataset to demonstrate the analysis workflow.

In [ ]:
# Create synthetic single-cell data
np.random.seed(42)

n_cells = 1000
n_genes = 2000

# Simulate count data with negative binomial distribution
X = np.random.negative_binomial(5, 0.3, (n_cells, n_genes))

# Create AnnData object
adata = sc.AnnData(X)

# Add gene and cell names
adata.var_names = [f'Gene_{i:04d}' for i in range(n_genes)]
adata.obs_names = [f'Cell_{i:04d}' for i in range(n_cells)]

# Add some metadata
adata.obs['batch'] = np.random.choice(['Batch_A', 'Batch_B'], n_cells)
adata.obs['cell_type'] = np.random.choice(['Type_1', 'Type_2', 'Type_3'], n_cells)

print(f"Created AnnData object with {adata.n_obs} cells and {adata.n_vars} genes")
print(adata)

## 3. Quality Control and Preprocessing with Scanpy

In [ ]:
# Calculate QC metrics
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # mitochondrial genes
sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=False, inplace=True)

# Add number of genes per cell
adata.obs['n_genes'] = (adata.X > 0).sum(axis=1)

print("QC metrics calculated")
print(f"Mean genes per cell: {adata.obs['n_genes'].mean():.1f}")
print(f"Mean UMIs per cell: {adata.obs['total_counts'].mean():.1f}")

In [ ]:
# Preprocessing steps
print("Starting preprocessing...")

# Filter cells and genes
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

# Save raw data
adata.raw = adata

# Normalize to 10,000 reads per cell
sc.pp.normalize_total(adata, target_sum=1e4)

# Log transform
sc.pp.log1p(adata)

# Find highly variable genes
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

# Keep only highly variable genes
adata = adata[:, adata.var.highly_variable]

# Scale data
sc.pp.scale(adata, max_value=10)

print(f"After preprocessing: {adata.n_obs} cells, {adata.n_vars} highly variable genes")

## 4. Dimensionality Reduction and Clustering

In [ ]:
# Principal component analysis
sc.tl.pca(adata, svd_solver='arpack')

# Compute neighborhood graph
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

# Perform UMAP embedding
sc.tl.umap(adata)

# Leiden clustering
sc.tl.leiden(adata, resolution=0.5)

print("Dimensionality reduction and clustering completed")
print(f"Number of clusters found: {len(adata.obs['leiden'].unique())}")

In [ ]:
# Visualize results with scanpy
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

sc.pl.umap(adata, color='leiden', ax=axes[0], show=False, frameon=False)
axes[0].set_title('Leiden Clusters')

sc.pl.umap(adata, color='batch', ax=axes[1], show=False, frameon=False)
axes[1].set_title('Batch')

sc.pl.umap(adata, color='cell_type', ax=axes[2], show=False, frameon=False)
axes[2].set_title('Cell Type (Simulated)')

plt.tight_layout()
plt.show()

## 5. Using R for Advanced Visualization

Now let's transfer our data to R and create publication-quality plots using ggplot2.

In [ ]:
# Prepare data for R
umap_coords = adata.obsm['X_umap']
plot_df = pd.DataFrame({
    'UMAP1': umap_coords[:, 0],
    'UMAP2': umap_coords[:, 1],
    'leiden_cluster': adata.obs['leiden'].values,
    'batch': adata.obs['batch'].values,
    'cell_type': adata.obs['cell_type'].values,
    'n_genes': adata.obs['n_genes'].values
})

print(f"Prepared data frame with {len(plot_df)} cells for R visualization")
print(plot_df.head())

In [ ]:
%%R -i plot_df -w 12 -h 8

# Load required R libraries
library(ggplot2)
library(dplyr)

# Create a publication-quality UMAP plot
p1 <- ggplot(plot_df, aes(x = UMAP1, y = UMAP2, color = leiden_cluster)) +
    geom_point(size = 0.8, alpha = 0.7) +
    theme_classic() +
    theme(
        axis.text = element_text(size = 12),
        axis.title = element_text(size = 14),
        legend.title = element_text(size = 12),
        legend.text = element_text(size = 10),
        plot.title = element_text(size = 16, hjust = 0.5)
    ) +
    labs(
        title = "Single-Cell UMAP Visualization",
        x = "UMAP 1",
        y = "UMAP 2",
        color = "Leiden\nCluster"
    ) +
    guides(color = guide_legend(override.aes = list(size = 3)))

print(p1)

In [ ]:
%%R -w 14 -h 6

# Create a multi-panel plot
library(gridExtra)

# Plot by batch
p2 <- ggplot(plot_df, aes(x = UMAP1, y = UMAP2, color = batch)) +
    geom_point(size = 0.8, alpha = 0.7) +
    theme_classic() +
    labs(title = "Batch Effect", x = "UMAP 1", y = "UMAP 2", color = "Batch") +
    theme(plot.title = element_text(hjust = 0.5))

# Plot by number of genes
p3 <- ggplot(plot_df, aes(x = UMAP1, y = UMAP2, color = n_genes)) +
    geom_point(size = 0.8, alpha = 0.7) +
    scale_color_viridis_c() +
    theme_classic() +
    labs(title = "Number of Genes", x = "UMAP 1", y = "UMAP 2", color = "Genes") +
    theme(plot.title = element_text(hjust = 0.5))

# Combine plots
grid.arrange(p2, p3, ncol = 2)

## 6. Statistical Analysis with R

Let's perform some statistical analysis using R's powerful statistical functions.

In [ ]:
%%R

# Statistical summary by cluster
cluster_stats <- plot_df %>%
    group_by(leiden_cluster) %>%
    summarise(
        n_cells = n(),
        mean_genes = mean(n_genes),
        sd_genes = sd(n_genes),
        .groups = 'drop'
    )

print("Cluster Statistics:")
print(cluster_stats)

# ANOVA test for differences in gene count between clusters
anova_result <- aov(n_genes ~ leiden_cluster, data = plot_df)
print("\nANOVA Results:")
print(summary(anova_result))

In [ ]:
%%R -w 10 -h 6

# Create a boxplot showing gene count distribution by cluster
p4 <- ggplot(plot_df, aes(x = leiden_cluster, y = n_genes, fill = leiden_cluster)) +
    geom_boxplot(alpha = 0.7) +
    geom_jitter(width = 0.2, alpha = 0.3, size = 0.5) +
    theme_classic() +
    labs(
        title = "Gene Count Distribution by Cluster",
        x = "Leiden Cluster",
        y = "Number of Genes",
        fill = "Cluster"
    ) +
    theme(
        plot.title = element_text(hjust = 0.5, size = 14),
        axis.text = element_text(size = 12),
        axis.title = element_text(size = 12)
    )

print(p4)

## 7. Transferring Results Back to Python

We can also transfer results from R back to Python for further analysis.

In [ ]:
%%R -o cluster_stats_r

# This will transfer the cluster_stats data frame to Python
cluster_stats_r <- cluster_stats

In [ ]:
# Now we can use the R results in Python
print("Cluster statistics transferred from R:")
print(cluster_stats_r)
print(f"\nData type: {type(cluster_stats_r)}")

# Convert to pandas DataFrame if needed
cluster_stats_pd = pd.DataFrame(cluster_stats_r)
print(f"\nAs pandas DataFrame:")
print(cluster_stats_pd)

## 8. Summary

This notebook demonstrated:

1. **Setting up the environment** with both Python and R packages
2. **Creating and preprocessing single-cell data** using scanpy
3. **Performing dimensionality reduction and clustering** with scanpy
4. **Creating publication-quality visualizations** with R and ggplot2
5. **Statistical analysis** using R's statistical functions
6. **Seamless data transfer** between Python and R environments

This workflow demonstrates the power of combining Python's computational efficiency (scanpy) with R's statistical and visualization capabilities (ggplot2, statistical tests) in a single analysis pipeline.

### Next Steps

- Try this workflow with real single-cell RNA-seq data
- Explore more advanced scanpy functions for trajectory analysis
- Learn about spatial transcriptomics analysis with squidpy
- Experiment with different R packages for specialized analyses